[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rareylab/proteins_plus_examples/blob/main/notebooks/EDIAscorer_example.ipynb)


# EDIAscorer: Assessing the density fit of individual atoms

The EDIA-score can be used to assess how well individual atoms of a 3D structure model fit with the electron density. The EDIA-scores can be inspected and visualized on a per atom basis. By simply combining atom scores the overall density-support of molecular structure like ligands and parts of the protein can be determined. 

With EDIA protein structures and ligands can be assessed and filtered by their experimental electron density support. 


[Estimating Electron Density Support for Individual Atoms and Molecular Fragments in X-ray Structures
Agnes Meyder, Eva Nittinger, Gudrun Lange, Robert Klein, and Matthias Rarey
Journal of Chemical Information and Modeling 2017 57 (10), 2437-2447 ](https://doi.org/10.1021/acs.jcim.7b00391)

Note: NGLview triggers the Colab code snippet sidebar every time a structure is visualized. Don't close it but resize it. In addition, sometimes the NGL views stay white and no structure is shown. In this case just run the cell again.

Note: Visualization with NGLView in this notebook does not really work on colab. They do work locally in jupyter notebook. Different representations are shown at the same time because clear_representations seems not to work as intended. We are on it to fix the issue. For now you can download the respective PDB/ligand/density files and view them with tools like Chimera or PyMol locally.

In [1]:
# colab allow nglview plugin
from google.colab import output
output.enable_custom_widget_manager()

ModuleNotFoundError: No module named 'google'

In [ ]:
# colab install dependencies
!pip install biopython &>> output.log
!pip install nglview &>> output.log
!pip install rdkit-pypi &>> output.log

In [2]:
# imports
import io
import os
from pathlib import Path
import requests
import sys
import time
from urllib.parse import urljoin

from Bio.PDB import *
import nglview as nv

In [3]:
# constants
PROTEINS_PLUS_URL = 'https://proteins.plus/api/v2/'
UPLOAD = urljoin(PROTEINS_PLUS_URL, 'molecule_handler/upload/')
UPLOAD_JOBS = urljoin(PROTEINS_PLUS_URL, 'molecule_handler/upload/jobs/')
PROTEINS = urljoin(PROTEINS_PLUS_URL, 'molecule_handler/proteins/')
LIGANDS = urljoin(PROTEINS_PLUS_URL, 'molecule_handler/ligands/')
EDIA = urljoin(PROTEINS_PLUS_URL, 'ediascorer/')
EDIA_JOBS = urljoin(PROTEINS_PLUS_URL, 'ediascorer/jobs/')
ATOM_SCORES = urljoin(PROTEINS_PLUS_URL, 'ediascorer/scores/')
EBI_URL = 'https://www.ebi.ac.uk/pdbe/coordinates/files/'

In [4]:
#@title Utils functions to call API (unhide if you're interested)

# check server connection
try:
    response = requests.get(PROTEINS_PLUS_URL + '/')
except requests.ConnectionError as error:
    if 'Connection refused' in str(error):
        print('WARNING: could not establish a connection to the server', file=sys.stderr)
    raise

# EDIA coloring for the NGL
edia_color_func = '''
  this.atomColor = function(atom){
    if(atom.bfactor > 1) {
      return 0x000AFF;
    } else if(atom.bfactor > 0.85){
      return 0x3C00FF;
    } else if(atom.bfactor > 0.75) {
      return 0xB400FF;
    } else if(atom.bfactor > 0.65) {
      return 0xF000FF;
    } else if(atom.bfactor > 0.55) {
      return 0xFF00D2;
    } else if(atom.bfactor > 0.45) {
      return 0xFF009E;
    } else if(atom.bfactor > 0.25) {
      return 0xFF0066
    } else if(atom.bfactor >= 0) {
      return 0xFF0000;
    } else {
      return 0xFFFFFF;
    }
  };
'''
nv.color.ColormakerRegistry.add_scheme_func('ediacolor', edia_color_func)
    
def get_density_for_pdbcode(pdb_code):
    """Downloads electron density file in ccp4 format for PDB code.
      
    :param pdb_code: The PDB code, like 1g9v
    :type pdb_code: str
    :return: filepath to downloaded density file.
    :rtype: str
    """
    req = requests.get(urljoin(EBI_URL, f'{pdb_code}.ccp4'))
    if req.status_code != 200:
      raise RuntimeError(f'Failed to retrieve density for {pdb_code}\n'
                          f'{req.text}')
    density_file = f'{pdb_code}.ccp4'
    with open(density_file, 'wb') as file:
        file.write(bytearray(req.content))
    return density_file

def poll_job(job_id, poll_url, poll_interval=1, max_polls=10):
    """Poll the progress of a job
    
    Continuosly polls the server in regular intervals and updates the job information, especially the status.
    
    :param job_id: UUID of the job to poll
    :type job_id: str
    :param poll_url: URl to send the polling request to
    :type poll_url: str
    :param poll_interval: time interval between polls in seconds
    :type poll_interval: int
    :param max_polls: maximum number of times to poll before exiting
    :type max_polls: int
    :return: polled job
    :rtype: dict
    """
    job = requests.get(poll_url + job_id + '/').json()
    status = job['status']
    current_poll = 0
    while status == 'pending' or status == 'running':
        print(f'Job {job_id} is { status }')
        current_poll += 1
        if current_poll >= max_polls:
            print(f'Job {job_id} has not completed after {max_polls} polling requests' \
                  f' and {poll_interval * max_polls} seconds')
            return job
        time.sleep(poll_interval)
        job = requests.get(poll_url + job_id + '/').json()
        status = job['status']
    print(f'Job {job_id} completed with { status }')
    return job

EDIAscorer calculates atomwise electron density support scores. A full introduction into the method can be found in this [publication](https://doi.org/10.1021/acs.jcim.7b00391). Let's take a look at the electron density for a PDB entry.

In [5]:
# fetch the protein 4agm from the PDB
file_4agm = Path(PDBList().retrieve_pdb_file('4agm', file_format='pdb'))
os.rename(file_4agm, '4agm.pdb')
file_4agm = '4agm.pdb' # ProteinsPlus needs .pdb extension

# download the corresponding electron density file
density_4agm = get_density_for_pdbcode('4agm')

# build a biopython protein
protein_structure = PDBParser().get_structure('4agm', '4agm.pdb')

/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3813.
  PDBConstructionWarning,
/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3835.
  PDBConstructionWarning,
/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3857.
  PDBConstructionWarning,
/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4161.
  PDBConstructionWarning,


Visualize the structure and it's density. 

(Some of these nglviews are a bit unstable due to the amount of data involved. If you only see a white box try to execute the cells again and give them some time to load.)

In [7]:
with open(density_4agm, 'rb') as density_file:
    view = nv.show_file(density_file, ext='ccp4')

In [8]:
view.add_component(nv.BiopythonStructure(protein_structure))
view

NGLWidget()

In [9]:
# Give it a few seconds to load. If the view above is a white box execute the previous two snippets and this one again.
view.clear_representations()
view.add_surface(wrap=True, contour=True, color="skyblue", opacity=0.8)

As you can see parts of the ligands are supported by electron density and parts aren't. Let's use the EDIA to quantify this.

In [10]:
with open('4agm.pdb') as upload_file:
    query = {'protein_file': upload_file}
    params = {'pdb_code': '4agm'}
    edia_job_submission = requests.post(EDIA, data=params, files=query).json()
# EDIA jobs can run for a long time. This may take 10 Minutes.
edia_job = poll_job(edia_job_submission['job_id'], EDIA_JOBS, poll_interval=5, max_polls=1000)

Job 3d280ee3-ac7e-4214-a69f-3c978ef85f5f completed with success


The call above sent the PDB file of 4AGM and a PDB code to the server. The PDB code is used to retrieve the density file. We could have also sent a local density file like this:

In [ ]:
# with open(TEST_FILES / '4agm.pdb') as upload_file:
#     with open(TEST_FILES / '4agm.ccp4', 'rb') as density_file:
#         query = {'protein_file': upload_file, 'electron_density_map': density_file}
#         edia_job_submission = requests.post(EDIA, files=query).json()
# # EDIA jobs can run for a long time. This may take 10 Minutes.
# edia_job = poll_job(edia_job_submission['job_id'], EDIA_JOBS, poll_interval=5, max_polls=1000)

The EDIAscorer produces two outputs the "edia_scores" and the "output_protein". The EDIA scores contain the atom scores, a large map of EDIA values for every atom in the protein. These can be mapped back to protein atoms of the output protein.

In [11]:
edia_scores = requests.get(ATOM_SCORES + edia_job['edia_scores'] + '/').json()
atom_scores = edia_scores['atom_scores']
output_protein = requests.get(PROTEINS + edia_job['output_protein'] + '/').json()

print('Residue\tChain\tAtom\tEDIA')
# going line by line in the protein
for line in output_protein['file_string'].split('\n'):
    if 'P86' in line:  # looking for the ligands
        # get the atom serial number
        atom_serial_number = line.split(' ')[1]
        # use the atom serial number (as a string) to index the atom scores
        atom_info = atom_scores[atom_serial_number]
        print('\t'.join([atom_info['Substructure name'], atom_info['Chain'], atom_info['Atom name'], atom_info['EDIA']]))
        

Residue	Chain	Atom	EDIA
P86	A	C01	1.04
P86	A	C1	1.06
P86	A	N1	1.11
P86	A	O1	1.03
P86	A	C02	0.97
P86	A	C2	1
P86	A	N2	0.88
P86	A	C03	0.93
P86	A	C3	0.96
P86	A	C4	0.79
P86	A	C7	1
P86	A	C8	0.93
P86	A	C10	0.95
P86	A	C11	0.46
P86	A	C12	0
P86	A	C13	0.79
P86	A	C14	0.59
P86	A	C15	0.89
P86	A	C16	0.85
P86	A	I1	1.09
P86	A	I2	1.16
P86	B	C01	1.03
P86	B	C1	1.08
P86	B	N1	1.11
P86	B	O1	1.06
P86	B	C02	0.93
P86	B	C2	0.99
P86	B	N2	0.83
P86	B	C03	0.98
P86	B	C3	1.02
P86	B	C4	0.79
P86	B	C7	1.08
P86	B	C8	0.92
P86	B	C10	0.94
P86	B	C11	0.31
P86	B	C12	0.32
P86	B	C13	0.71
P86	B	C14	0.43
P86	B	C15	0.94
P86	B	C16	0.94
P86	B	I1	0.95
P86	B	I2	1.17


EDIA values < 0.8 are minor inconsistencies, whereas EDIA values < 0.4 are substantial inconsistencies (more information in the [publication](https://doi.org/10.1021/acs.jcim.7b00391)). The raw values are helpful, but let's visualize the support for each atom:

In [12]:
output_protein_file = io.StringIO(output_protein['file_string'])
protein_structure = PDBParser().get_structure(output_protein['name'], output_protein_file)
view = nv.show_biopython(protein_structure, color='ediacolor')
view.clear_representations()
view.add_cartoon(color='ediacolor')
view.add_representation(repr_type='ball+stick', selection='ligand', color='ediacolor')
view

/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3103.
  PDBConstructionWarning,
/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3125.
  PDBConstructionWarning,
/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3147.
  PDBConstructionWarning,
/ssd_local/penner/miniconda/envs/examples/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3451.
  PDBConstructionWarning,


NGLWidget()

The gradient purple to red shows well supported to poorly supported atoms. The red atoms of the ligands have low EDIA values in the raw value list and have little electron density associated with them in the above visualization of the electron density map.

There is one more type of output information: EDIAm scores for all residues (and ligands) of a complex. These are summary EDIA scores for molecules. You can find these in the EDIA scores as "structure_scores". EDIAm scores of < 0.8 should raise concern. In this case we're seeing the flexible substituent that reaches outside of the pocket not being very well supported.

In [13]:
print('Residue\tChain\tEDIAm\tMin EDIA\tMax EDIA')
structure_scores = edia_scores['structure_scores']
for residue in structure_scores.keys():
    if 'P86' in residue:
        ligand_scores = structure_scores[residue]
        print('\t'.join([
            ligand_scores['Name'],
            ligand_scores['Chain'],
            ligand_scores['EDIAm'],
            ligand_scores['Min EDIA'],
            ligand_scores['Max EDIA']
        ]))

Residue	Chain	EDIAm	Min EDIA	Max EDIA
P86	A	0.32	0	1.16
P86	B	0.72	0.31	1.17
